# Notebook 10: YOLO General

En este caso se va a realizar un notebook general que permita crear un modelo de detección, para que cualquier usuario pueda usarlo dado un conjunto de datos cualquiera.

En primer lugar, vamos a definir unos parámetros y cargar la librería donde se encuentran las funciones que nos van a permitir realizar los pasos necesarios para configurar y entrenar la red YOLO.

In [0]:
from funciones import *

En el siguiente paso indicamos cuál va a ser el nombre del proyecto:

In [0]:
DATASET = "estomas2"

Después introducimos la ruta donde se ha descargado la librería darknet. La siguiente instrucción nos indica donde se encuentra la librería.

In [0]:
!pwd

/home/alumno/Descargas


In [0]:
DARKNET = "/home/alumno/Descargas/darknet/"

### 1- Recolección del dataset.

Para entrenar YOLO nos hace falta un conjunto de imágenes que formen el dataset, la única condición que se pide aquí es que las imágenes estén en formato *.jpg*. Todas las imágenes en formato *.jpg* deben estar almacenadas dentro de una carpeta y la ruta de dicha carpeta debe introducirse en la siguiente variable.

In [0]:
PATHIMAGES = "/home/alumno/images"

La siguiente instrucción permite validar que la ruta anterior es válida.

In [0]:
compruebeImages(PATHIMAGES)

### 2. Anotación del dataset.

Necesitamos que todas las imágenes estén anotadas en el formato YOLO, es decir YOLO necesita que cada imagen tenga un archivo .txt con una  línea por cada objeto que nos interese detectar. Para generar este tipo de archivos podemos usar programas como Yolo_mark.
Necesitamos que todas las imágenes estén anotadas y todos los ficheros *.txt*  con las anotaciones deben estar almacenados en la misma carpeta donde se han almacenado las imágenes.

La siguiente función comprueba que cada imagen tiene su fichero con la anotación.

In [0]:
compruebeTXT(PATHIMAGES)

El siguiente paso consiste en crear el fichero *.names* con las clases de objetos que se quieren detectar. Para ello necesitamos la lista de las clases que vamos a detectar, el nombre del proyecto (que dará lugar al nombre del fichero) y la ruta donde se encuentra la darknet. 

Ahora añadimos en la siguiente variable la lista de las clases de objetos que quieres que se detecte.

In [0]:
CLASSES='perro,gato, pájaro'
numClases= contarClases(CLASSES)

Generamos el fichero:

In [0]:
generaFicheroNames(DARKNET, DATASET, CLASSES)

### 3- Data augmentation.

Los pasos 2 y 3 son intercambiables, en el resto de notebooks primero se realizaba el *data augmentation* y luego la *anotación del dataset*. En cambio en este notebook se ha decidido realizar primero la *anotación del dataset* y después el *data augmentation*.

En caso de que el dataset sea pequeño, será necesario aumentarlo aplicándole al conjunto inicial de imágenes una serie de filtros, rotaciones, etc. Se puede usar una librería desarrollada por la Universidad de La Rioja, llamada CLoDSA. Esta librería permite aplicar un conjunto de transformaciones y a la vez generar de manera automática las anotaciones correspondientes. De este modo, sólo tienes que anotar el conjunto de imágenes inicial, y la librería se encarga de manera automática de generar las imágenes aumentadas y sus correspondientes anotaciones, evitando la anotación manual de muchas imágenes. Se pueden realizar transformaciones como las siguientes: 
* Volteo vertical
* Volteo horizontal 
* Volteo horizontal y vertical
* Rotaciones de 90, 180 y 270 grados
* El filtro de la media
* Ruido gausiano

Para aplicar estas técnicas ir al notebook **CLODSA_Estomas**, donde se puede ver un ejemplo de cómo aplicarlo.

### 4- Dataset split

Como en cualquier modelo de aprendizaje automático, es muy importante descomponer el conjunto de datos en dos partes:
* El conjunto de entrenamiento: conjunto de imágenes que utilizaremos para el entrenamiento del algoritmo.
* En conjunto de evaluación: conjunto de imágenes que utilizaremos para la evaluación del  algoritmo.

La idea de este apartado es dividir el conjunto de imágenes, en un conjunto de entrenamiento y otro conjunto de test utilizando como partición una de las de la siguiente figura (son los más habituales).
<img src="imágenes/datasetSplitPorcentajes.png" style="width:750px;">

Lo primero que hay que hacer en este apartado es decidir el porcentaje de división:

In [0]:
PORCENTAJE_ENTRENAMIENTO = 0.75

Realizamos la separación de los datos en dos conjuntos, el de entrenamiento y el de test. Con la siguiente instrucción se moverán las imágenes y sus anotaciones a las carpetas correspondientes y cuya estructura es muy relevante para que la red YOLO se pueda entrenar.

In [0]:
datasetSplit( DATASET, DARKNET, PATHIMAGES, PORCENTAJE_ENTRENAMIENTO)

Una vez realizado el *dataset split* podemos crear los ficheros train.txt y test.txt, donde tendremos almacenas las rutas de las imágenes de ambos conjuntos.

In [0]:
generaFicheroTrain(DARKNET, DATASET)
generaFicheroTest(DARKNET, DATASET)

### 5- Entrenamiento

El primer paso antes de empezar a entrenar será crear el archivo de configuración *.data* para que apunte a las imágenes que queremos utilizar. 

In [0]:
generaFicheroData(DARKNET, numClases,DATASET)

A parte de este fichero también es necesario crear el fichero *.cfg*, indicando que vamos a realizar el entrenamiento.

In [0]:
generaFicherosYoloTrain(DARKNET, DATASET, numClases)

Hay dos maneras de entrenar la red YOLO: o bien usando unos pesos pre-entrenados o partiendo de cero. Si quieres que parta de cero ponemos a True la variable desdeCero, en caso contrario False.

In [0]:
desdeCero = True

In [0]:
generaInstruccionEntrenar(DATASET, desdeCero)

Ejecuta la instrucción anterior.

### 6- Evaluación

Una vez finalizado el entrenamiento, y antes de comprobar que nos funciona en imágenes reales, vamos a evaluar el modelo construido en el conjunto de test.

Hace falta crear un fichero *.cfg*, indicando que vamos a realizar la evaluación.

In [0]:
generaFicherosYoloTest(DARKNET, DATASET, numClases)

En la evaluación de un modelo se puede elegir el nivel de confianza al que se quiere evaluar, por defecto es de 0.25 (un 25%), si se quiere evaluar con otro añadir a la siguiente función un tercer parámetro indicando el nivel de confianza. La siguiente instrucción muestra todos los pesos generados en el entrenamiento.

In [0]:
listarBackup(DARKNET)

Seleccionamos un fichero de pesos de la lista anterior y lo ponemos en la siguiente instrucción.

In [0]:
generaInstruccionEvaluar(DATASET, pathPesos)

no existen esos pesos


Ejecuta la instrucción anterior.

### 7- Predicción
Por último podemos probar el modelo con imágenes que no se encuentren en el conjunto inicial. Ejecuta para ello la siguiente instrucción. Igual que en la evaluación para la predicción se puede modificar el nivel de confianza, por defecto sera de 0.25 pero si quieres modificarlo añade un parámetro más a la función

In [0]:
generaInstruccionPredecir(DATASET, pathPesos, pathFoto):

Ejecuta la instrucción anterior.